In [10]:
import os
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium.spaces import Box, MultiDiscrete
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# Function to load and preprocess electric data
def load_and_preprocess_electric_data(house_dir):
    # List CSV files in the electric data directory
    electric_dir = os.path.join(house_dir, 'Electric_data')
    csv_files = [f for f in os.listdir(electric_dir) if f.endswith('.csv')]
    dfs = []
    for csv_file in csv_files:
        file_path = os.path.join(electric_dir, csv_file)
        df = pd.read_csv(file_path)
        print(df.columns)
        print(csv_file)
        # Parse timestamp
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)
        dfs.append(df)
    # Concatenate all months
    electric_df = pd.concat(dfs)
    # Resample to 1-minute intervals
    electric_df = electric_df.resample('1T').mean()
    # Forward-fill small gaps (up to 2 missing intervals, i.e., 30 seconds)
    electric_df = electric_df.ffill(limit=2)
    # Handle missing values
    electric_df = electric_df.dropna()
    return electric_df

# Function to load and preprocess environmental data
def load_and_preprocess_environmental_data(house_dir):
    # Environmental data directory
    env_dir = os.path.join(house_dir, 'Environmental_data')
    csv_files = [f for f in os.listdir(env_dir) if f.endswith('.csv')]
    dfs = []
    for csv_file in csv_files:
        file_path = os.path.join(env_dir, csv_file)
        df = pd.read_csv(file_path)
        # Parse timestamp
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)
        dfs.append(df)
    # Concatenate all months
    env_df = pd.concat(dfs)
    # Resample to 1-minute intervals
    env_df = env_df.resample('1T').interpolate()
    # Forward-fill small gaps (up to 60 minutes)
    env_df = env_df.ffill(limit=60)
    # Handle missing values
    env_df = env_df.dropna()
    return env_df

# Function to load socio-economic data
def load_socio_economic_data(house_dir):
    # Socio-economic data file
    socio_file = os.path.join(house_dir, 'Sociodemographic', 'socioeconomic_data.csv')
    df = pd.read_csv(socio_file)
    # Convert to dictionary
    socio_data = df.to_dict(orient='records')[0]  # Assuming one record per house
    return socio_data

# Local Home Energy Management System Environment
class LHEMSEnv(gym.Env):
    metadata = {
        "render_modes": ["human"],
    }

    def __init__(self, electric_data, env_data, appliances_metadata, comfort_prefs, socio_data):
        super().__init__()
        self.electric_data = electric_data
        self.env_data = env_data
        self.appliances_metadata = appliances_metadata
        self.socio_data = socio_data  # Socio-economic data

        self.appliance_names = [col for col in electric_data.columns if col not in ['V', 'A', 'P_agg', 'issues']]
        self.num_appliances = len(self.appliance_names)
        self.current_time_index = 0
        self.max_time_steps = len(electric_data)

        # Adjust comfort preferences based on socio-economic data
        self.comfort_prefs = self.adjust_comfort_prefs(comfort_prefs)

        # Define action space: Each appliance can be ON or OFF
        self.action_space = gym.spaces.MultiDiscrete([2]*self.num_appliances)

        # Define observation space
        # For each appliance: [Normalized Power, Desired State]
        # Environmental data: [Internal Temp, Internal Humidity, External Temp, External Humidity]
        # Time Features: [Hour, Day of Week]
        obs_low = [0.0]*(2*self.num_appliances) + [-50.0]*4 + [0.0, 0.0]
        obs_high = [1.0]*(2*self.num_appliances) + [50.0]*4 + [1.0, 1.0]
        self.observation_space = gym.spaces.Box(
            low=np.array(obs_low, dtype=np.float32),
            high=np.array(obs_high, dtype=np.float32),
            dtype=np.float32
        )

    def adjust_comfort_prefs(self, base_prefs):
        adjusted_prefs = base_prefs.copy()

        # Example adjustments based on socio-economic data

        # Adjust temperature preferences based on age
        age = self.socio_data.get('Age of the respondent', 35)
        if age >= 65:
            # Older individuals may prefer warmer temperatures
            adjusted_prefs['temperature_range'] = [22.0, 27.0]
        elif age <= 25:
            # Younger individuals may prefer cooler temperatures
            adjusted_prefs['temperature_range'] = [18.0, 23.0]
        else:
            # Default temperature range
            adjusted_prefs['temperature_range'] = base_prefs.get('temperature_range', [20.0, 25.0])

        # Adjust cost sensitivity based on income
        income_bracket = self.socio_data.get('Family monthly income', 'Medium')
        if income_bracket == 'High':
            # Less sensitive to energy cost
            adjusted_prefs['cost_weight'] = 0.5
            adjusted_prefs['comfort_weight'] = 1.5
        elif income_bracket == 'Low':
            # More sensitive to energy cost
            adjusted_prefs['cost_weight'] = 1.5
            adjusted_prefs['comfort_weight'] = 0.5
        else:
            # Default weights
            adjusted_prefs['cost_weight'] = 1.0
            adjusted_prefs['comfort_weight'] = 1.0

        # Adjust comfort preferences if pets are present
        pets = self.socio_data.get('Pets', 'No pets')
        if pets != 'No pets':
            # Increase comfort weight
            adjusted_prefs['comfort_weight'] += 0.5

        return adjusted_prefs

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_time_index = 0
        self.done = False
        observation = self._get_obs()
        info = {}
        return observation, info

    def step(self, action):
        self._take_action(action)
        observation = self._get_obs()
        reward = self._calculate_reward(action)
        terminated = self._is_terminated()
        truncated = False
        info = {}

        self.current_time_index += 1
        return observation, reward, terminated, truncated, info

    def _get_obs(self):
        obs = []
        timestamp = self.electric_data.index[self.current_time_index]

        # Appliance observations
        for appliance_name in self.appliance_names:
            power = self.electric_data[appliance_name].iloc[self.current_time_index]
            # Normalize power based on appliance metadata
            max_power = self.appliances_metadata.get(appliance_name, {}).get('cutoff', 1.0)
            normalized_power = power / max_power if max_power > 0 else 0.0
            desired_state = 1 if normalized_power > 0.1 else 0
            obs.extend([normalized_power, desired_state])

        # Environmental data
        internal_temp = self.env_data['internal_temperature'].iloc[self.current_time_index]
        internal_humidity = self.env_data['internal_humidity'].iloc[self.current_time_index]
        external_temp = self.env_data['external_temperature'].iloc[self.current_time_index]
        external_humidity = self.env_data['external_humidity'].iloc[self.current_time_index]
        obs.extend([internal_temp, internal_humidity, external_temp, external_humidity])

        # Time features
        hour = timestamp.hour / 23.0  # Normalize to [0,1]
        day_of_week = timestamp.dayofweek / 6.0  # Normalize to [0,1]
        obs.extend([hour, day_of_week])

        return np.array(obs, dtype=np.float32)

    def _take_action(self, action):
        # Update appliance states based on action
        self.actions = action

    def _calculate_reward(self, action):
        total_reward = 0
        timestamp = self.electric_data.index[self.current_time_index]

        # Environmental comfort parameters
        internal_temp = self.env_data['internal_temperature'].iloc[self.current_time_index]
        desired_temp_range = self.comfort_prefs.get('temperature_range', [20.0, 25.0])
        temp_penalty = self.comfort_prefs.get('temp_penalty', 5.0)

        # Cost and comfort weights
        cost_weight = self.comfort_prefs.get('cost_weight', 1.0)
        comfort_weight = self.comfort_prefs.get('comfort_weight', 1.0)

        for idx, appliance_name in enumerate(self.appliance_names):
            power = self.electric_data[appliance_name].iloc[self.current_time_index]
            max_power = self.appliances_metadata.get(appliance_name, {}).get('cutoff', 1.0)
            normalized_power = power / max_power if max_power > 0 else 0.0
            desired_state = 1 if normalized_power > 0.1 else 0
            actual_state = action[idx]
            price = 1.0  # Modify as needed

            # Negative electric cost
            energy_cost = -cost_weight * price * normalized_power * actual_state

            # Comfort penalty
            comfort_penalty = 0

            # Appliance-specific comfort preferences
            appliance_prefs = self.comfort_prefs.get(appliance_name, {})
            if desired_state == 1 and actual_state == 0:
                comfort_penalty += comfort_weight * appliance_prefs.get('penalty', 10.0)

            # For AC units, consider temperature comfort
            if appliance_name.startswith('ac') and actual_state == 0:
                if internal_temp > desired_temp_range[1]:
                    comfort_penalty += comfort_weight * temp_penalty * (internal_temp - desired_temp_range[1])
                elif internal_temp < desired_temp_range[0]:
                    comfort_penalty += comfort_weight * temp_penalty * (desired_temp_range[0] - internal_temp)

            # Total reward for the appliance
            reward = energy_cost - comfort_penalty
            total_reward += reward

        return total_reward

    def _is_terminated(self):
        return self.current_time_index >= self.max_time_steps - 1

    def render(self):
        pass

    def close(self):
        pass

# Function to train local models for each LHEMS
def train_local_model(env, global_params=None, total_timesteps=10000):
    # Initialize the local model
    model = PPO('MlpPolicy', env, verbose=0)
    if global_params:
        model.policy.load_state_dict(global_params)
    # Train the model
    model.learn(total_timesteps=total_timesteps)
    # Return the trained model parameters
    return model.policy.state_dict()

# Function to aggregate global model parameters
def aggregate_global_model(local_models_params):
    global_params = {}
    num_models = len(local_models_params)
    for key in local_models_params[0].keys():
        # Average the parameters across all local models
        global_params[key] = sum([model_params[key] for model_params in local_models_params]) / num_models
    return global_params

# Federated training loop
def federated_training(lhems_envs, num_rounds=5, local_timesteps=10000):
    # Initialize local models
    local_models = [None] * len(lhems_envs)
    global_params = None

    for round in range(num_rounds):
        print(f"--- Federated Training Round {round+1}/{num_rounds} ---")
        local_models_params = []
        # Local training at each LHEMS
        for idx, env in enumerate(lhems_envs):
            print(f"Training local model for LHEMS {idx+1}")
            local_params = train_local_model(env, global_params=global_params, total_timesteps=local_timesteps)
            local_models_params.append(local_params)
        # Global aggregation at GS
        global_params = aggregate_global_model(local_models_params)
        print("Global model aggregated and updated.")

    # Return the final global model parameters
    return global_params

# Example usage
if __name__ == "__main__":
    # Directory containing house folders
    data_dir = r"C:\Users\hboki\OneDrive - ku.ac.ae\plegmaDataset_clean\Clean_Dataset"
    house_dirs = [os.path.join(data_dir, d) for d in os.listdir(data_dir) if d.startswith('House')]

    # Base comfort preferences
    base_comfort_prefs = {
        'ac_1': {'penalty': 5.0},
        'ac_2': {'penalty': 5.0},
        'boiler': {'penalty': 10.0},
        'fridge': {'penalty': 8.0},
        'washing_machine': {'penalty': 6.0},
        'temperature_range': [20.0, 25.0],  # Default desired indoor temperature range
        'temp_penalty': 5.0,  # Penalty per degree outside desired range
        'cost_weight': 1.0,   # Default cost weight
        'comfort_weight': 1.0, # Default comfort weight
    }

    # Appliances metadata (e.g., from appliances_metadata.csv)
    appliances_metadata = {
        'ac_1': {'cutoff': 2000.0},
        'ac_2': {'cutoff': 2000.0},
        'boiler': {'cutoff': 1500.0},
        'fridge': {'cutoff': 300.0},
        'washing_machine': {'cutoff': 1000.0},
        # Add more appliances as needed
    }

    # Create LHEMS environments for each house
    lhems_envs = []
    for house_dir in house_dirs:
        # Load electric data
        electric_data = load_and_preprocess_electric_data(house_dir)
        # Load environmental data
        env_data = load_and_preprocess_environmental_data(house_dir)
        # Load socio-economic data
        socio_data = load_socio_economic_data(house_dir)
        # Create environment
        env = LHEMSEnv(electric_data, env_data, appliances_metadata, base_comfort_prefs, socio_data)
        lhems_envs.append(env)

    # Run federated training
    final_global_params = federated_training(lhems_envs, num_rounds=5, local_timesteps=10000)
    print("Federated training completed.")

    # Save the final global model parameters
    print("Final global model parameters are ready for use.")


Index(['timestamp', 'V', 'A', 'P_agg', 'ac_1', 'ac_2', 'boiler', 'fridge',
       'washing_machine', 'issues'],
      dtype='object')
2022-07.csv
Index(['timestamp', 'V', 'A', 'P_agg', 'ac_1', 'ac_2', 'boiler', 'fridge',
       'washing_machine', 'issues'],
      dtype='object')
2022-08.csv
Index(['timestamp', 'V', 'A', 'P_agg', 'ac_1', 'ac_2', 'boiler', 'fridge',
       'washing_machine', 'issues'],
      dtype='object')
2022-09.csv
Index(['timestamp', 'V', 'A', 'P_agg', 'ac_1', 'ac_2', 'boiler', 'fridge',
       'washing_machine', 'issues'],
      dtype='object')
2022-10.csv
Index(['timestamp', 'V', 'A', 'P_agg', 'ac_1', 'ac_2', 'boiler', 'fridge',
       'washing_machine', 'issues'],
      dtype='object')
2022-11.csv
Index(['timestamp', 'V', 'A', 'P_agg', 'ac_1', 'ac_2', 'boiler', 'fridge',
       'washing_machine', 'issues'],
      dtype='object')
2022-12.csv
Index(['timestamp', 'V', 'A', 'P_agg', 'ac_1', 'ac_2', 'boiler', 'fridge',
       'washing_machine', 'issues'],
      dtype=

KeyError: 'timestamp'

In [2]:
!pip install gymnasium
!pip install stable-baselines3


   ---------------------------------------- 0.0/953.9 kB ? eta -:--:--
   ---------------------------------------- 10.2/953.9 kB ? eta -:--:--
   --------- ------------------------------ 215.0/953.9 kB 4.4 MB/s eta 0:00:01
   --------------------------------------  952.3/953.9 kB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 953.9/953.9 kB 6.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/182.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/182.3 kB ? eta -:--:--
   -- ------------------------------------- 10.2/182.3 kB ? eta -:--:--
   ------ -------------------------------- 30.7/182.3 kB 435.7 kB/s eta 0:00:01
   ------------------- ------------------- 92.2/182.3 kB 871.5 kB/s eta 0:00:01
   -------------------------------------- - 174.1/182.3 kB 1.2 MB/s eta 0:00:01
   -------------------------------------- 182.3/182.3 kB 915.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------